# ✏️ English Writing Feedback Tool (GPT-4o)
Click **'🔁 New Prompts'** to shuffle 3 story, 3 essay, and 3 opinion prompts each time.
Then choose one and write 150–200 words.

**Scoring (out of 40 points):**
- Spelling (10)
- Grammar (10)
- Vocabulary (10)
- Structure & Cohesion (10)

⚠️ Slight penalty for writing under 150 or over 260 words.

In [ ]:
!pip install openai==0.28 gradio --quiet

In [ ]:
import openai
import gradio as gr
import random

openai.api_key = input("Paste your OpenAI API key (starts with sk-...): ").strip()

In [ ]:
def shuffle_prompts(dummy=None):
    story_prompts = [
        ("A day when everything went hilariously wrong", "You wake up to find out you’ve swapped lives with your pet."),
        ("The secret door behind the bookshelf", "You step through and land in a parallel version of your school."),
        ("The robot that wouldn’t obey", "A school project turns into a chaotic adventure.")
    ]
    essay_prompts = [
        ("Describe the benefits and drawbacks of online learning", "Compare how students learn at home vs in the classroom."),
        ("How technology affects friendships", "Explain how devices change how kids connect."),
        ("Is it better to work in teams or alone?", "Use examples from school to support your view.")
    ]
    opinion_prompts = [
        ("Should schools require students to wear uniforms?", "Argue why you agree or disagree."),
        ("Should students have homework every day?", "Support your opinion with reasons."),
        ("Is it fair to ban junk food at school?", "Give your thoughts based on personal experience.")
    ]

    random.shuffle(story_prompts)
    random.shuffle(essay_prompts)
    random.shuffle(opinion_prompts)

    def format_section(title, prompts):
        lines = [f"**{title}**"]
        for i, (prompt, example) in enumerate(prompts, 1):
            lines.append(f"{i}. *{prompt}*  ")
            lines.append(f"   _Example_: {example}\n")
        return '\n'.join(lines)

    return (
        '### ✍️ Writing Prompts (Choose One)\n\n'
        + format_section("Story", story_prompts)
        + '\n' + format_section("Essay", essay_prompts)
        + '\n' + format_section("Opinion", opinion_prompts)
    )

In [ ]:
def get_feedback(essay_text):
    grading_prompt = f"""You are an English teacher grading a student writing task.

{essay_text}

Evaluate the following 4 criteria out of 10 points each (total 40 points):
1. Spelling – Are words spelled correctly?
2. Grammar – Are verb tenses and sentence structures correct?
3. Language Repertoire – Is vocabulary varied and appropriate?
4. Formulation – Is the text well structured with cohesive devices?

Also, apply a small penalty if the word count is under 150 or over 260 words.

Then:
- Provide brief feedback for each criterion
- Show individual scores and final score out of 40 (after word length penalty)
- Keep the tone constructive and encouraging.
"""
    try:
        response = openai.ChatCompletion.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": grading_prompt}],
            temperature=0.6,
            max_tokens=500
        )
        return response['choices'][0]['message']['content']
    except Exception as e:
        return f"⚠️ Error: {str(e)}"

In [ ]:
with gr.Blocks() as app:
    gr.Markdown("## English Writing Feedback Tool (Scored on 40 Points)")
    with gr.Row():
        prompt_box = gr.Markdown("Click 🔁 New Prompts to begin.")
        new_prompt_btn = gr.Button("🔁 New Prompts")
    new_prompt_btn.click(fn=shuffle_prompts, inputs=None, outputs=prompt_box)

    writing_input = gr.Textbox(label="Paste your writing here", lines=15)
    feedback_output = gr.Textbox(label="GPT Feedback")
    submit_btn = gr.Button("✉️ Submit Writing")
    submit_btn.click(fn=get_feedback, inputs=writing_input, outputs=feedback_output)

app.launch()